# Designing neuronal networks

## Overview

This notebook provides a guide to specyfing neural interconnections and designing networks using Hodgkin-Huxley and NeuroDyn models in Python environment.

## Setting up the environment

Let's first import the required Python modules and model definitions:

In [1]:
import matplotlib.pyplot as plt
from cb_models import HHModel, NeuroDynModel

# **Ignore overflow warnings**
import numpy as np
np.warnings.filterwarnings('ignore', 'overflow')

%matplotlib inline
%load_ext autoreload
%autoreload 2

### Neural interconnections

There are two main types of neural interconnections that we can implement: gap junctions and synaptic connections. In the conductance-based modelling framework, these interconnections lead to additional currents in the membrane equation of neuron $i$ that are dependent on membrane voltages of other neurons $j$ connected to it:

\begin{equation}
C \dot{V}_i = - I_{int,i} - \sum_{j} I_{gap, i, j} - \sum_{j} I_{syn, i, j} + I_{app}
\end{equation}

#### Gap junctions

Gap junctions are modeled as simple resistive connections between neurons, so that a resistive connection between neurons $i$ and $j$ leads to a current in neuron $i$:

\begin{equation}
I_{gap, i, j} = g_{gap, i, j} (V_i - V_j)
\end{equation}

Since these interconnections are symmetric $g_{gap, i, j} = g_{gap, j, i}$, so that:

\begin{equation}
I_{gap, j, i} = -I_{gap, i, j}
\end{equation}

We can represent this as a matrix $G_{gap}$ where each element $g_{gap, i, j}$ is the conductance of the connection between neurons $i$ and $j$. This matrix is necessarily symmetric, i.e. $G_{gap} = G^T_{gap}$.

#### Synaptic connections

Unlike gap junctions, synaptic connections are directed connections and are not symmetric: the activity of the *presynaptic* neuron (neuron $j$) modulates the activity of the *postsynaptic* neuron (neuron $i$). Synaptic currents have a similar form to the internal ionic currents, with the difference that their activation depends on the presynaptic neuron:

\begin{equation}
I_{syn, i, j} = \bar{g}_{syn, i, j} r (V_i - E_{syn}) \\
\dot{r} = \alpha_r(V_j) (1 - r) - \beta_r{V_i} r
\end{equation}

The opening of the channels $\alpha_r$ depends on the presynaptic neuron's voltage $V_j$, while the closing depends on the postsynaptic neuron's voltage $V_i$ (sometimes this relationship is modeled as constant).

The reversal potential of the synapse $E_{syn}$ determines if the synapse is *excitatory* or *inhibitory*: excitatory synapses tend to increase the postsynaptic neuron's voltage, while the inhibitory synapses act to decrease the postynaptic voltage. On the other hand, dynamics of activation $r$ determines the timescale of the synaptic current, i.e. how fast it activates in relation to the changes in the presynaptic neuron.

### Defining neural interconnections in Python

Let's now see how we can define these interconnections in the Python s